In [1]:
%matplotlib inline
import scipy.io
import numpy as np
import pandas as pd
from pathlib import Path
import re
from collections import Counter
from matplotlib import pyplot as plt
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
from torchvision import datasets, transforms
from torch.autograd import Variable
from torch.utils.data import TensorDataset
import torch.utils.data as data
from torchvision import datasets
device = torch.device("cuda" if torch.cuda.is_available() else "cpu") # use gpu

In [2]:
def import_depth_data(action, subject, trial):
    filename = f'../Data/UTDMHAD_data/Depth/a{action}_s{subject}_t{trial}_depth.mat'# The name of the .mat files
    if Path(filename).is_file():
        mat = scipy.io.loadmat(filename)
        return mat['d_depth']
    else:
        return None

def transform_depth_data(action, subject, trial):
    rows = []
    data = import_depth_data(action, subject, trial)
    if data is None: return None
    for frame in range(data.shape[2]):
        pixels = data[:, :, frame].flatten()
        rows.append(pixels)
    result = np.insert(rows, 0, [[action], [subject], [trial], [frame]], axis=1)
    return np.array(result)

def transform_depth_data_to_df(action, subject, trial):
    data = transform_depth_data(action, subject, trial)
    if data is None: return None
    df = pd.DataFrame(data)
    df.columns = ['action', 'subject', 'trial', 'frame'] + [f'depth_{n}' for n in range(240 * 320)]
    return df

def export_depth_data_to_csv(action, subject, trial):
    df = transform_depth_data_to_df(action, subject, trial)
    if df is None: return None
    filename = f'a{action}_s{subject}_t{trial}_depth.csv'
    df.to_csv(filename, index=False)

def show_depth_image(action, subject, trial, frame):
    data = import_depth_data(action, subject, trial)
    if data is None: return None
    plt.imshow(data[:,:,frame], cmap='gray')
    plt.axis('off')
    plt.show()

In [3]:
def import_inertial_data(action, subject, trial):
    filename = f'../Data/UTDMHAD_data/Inertial/a{action}_s{subject}_t{trial}_inertial.mat'
    if Path(filename).is_file():
        mat = scipy.io.loadmat(filename)
        return mat['d_iner']
    else:
        return None
    
def transform_inertial_data(action, subject, trial):
    data = import_inertial_data(action, subject, trial)
    if data is None: return None
    result = np.insert(data, 0, [[action], [subject], [trial]], axis=1)
    return np.array(result)

def transform_inertial_data_to_df(action, subject, trial):
    data = transform_inertial_data(action, subject, trial)
    if data is None: return None
    df = pd.DataFrame(data)
    df.columns = ['action', 'subject', 'trial', 'x-accel', 'y-accel', 'z-accel', 'x-ang-accel', 'y-ang-accel', 'z-ang-accel']
    return df

In [57]:
activities = [i for i in range(1,28)]
utd_df = transform_inertial_data_to_df(0, 0, 0)
for index, action in enumerate(activities):
    for subject in range(1, 9):
        for trial in range(1, 5):
            data = transform_inertial_data_to_df(action, subject, trial) # (160, 6)
            if data is None: continue
            #data = data[0:128] #(128.6) # maximum length is 128
            utd_df = pd.concat([utd_df, data])
            
# substract the label value by 1
utd_df['action'] -= 1 
utd_df

,action,subject,trial,x-accel,y-accel,z-accel,x-ang-accel,y-ang-accel,z-ang-accel
0,0.0,1.0,1.0,-0.959473,-0.177734,-0.192871,5.221374,1.526718,0.152672
1,0.0,1.0,1.0,-0.961914,-0.153320,-0.159912,6.778626,1.954198,0.244275
2,0.0,1.0,1.0,-0.974609,-0.152832,-0.145996,11.267176,3.175573,1.099237
3,0.0,1.0,1.0,-0.941895,-0.135742,-0.127930,16.885496,4.732824,2.320611
4,0.0,1.0,1.0,-0.958252,-0.201416,-0.139404,16.030534,4.000000,0.366412
...,...,...,...,...,...,...,...,...,...
250,26.0,8.0,3.0,-0.908936,0.045410,0.541992,-1.801527,0.427481,0.000000
251,26.0,8.0,3.0,-0.909668,0.055176,0.532471,-2.076336,0.335878,0.244275
252,26.0,8.0,3.0,-0.916748,0.050781,0.528320,-1.954198,0.396947,0.305344
253,26.0,8.0,3.0,-0.915039,0.041260,0.534424,-1.648855,0.244275,0.213740


In [35]:
# window_size: size of time window
# step: window_size * (100 - overlapping) / 100
# data: dataset
def time_windows(window_size,overlapping,dataset):
  sigmentation_data_temp = []
  sigmentation_data = []
  sigmentation_label = []
  step = int(window_size * (100 - overlapping) / 100)
  for i in range(0,len(dataset),step):
    acc_x = dataset['x-accel'].values[i:i+window_size]
    acc_y = dataset['y-accel'].values[i:i+window_size]
    acc_z = dataset['z-accel'].values[i:i+window_size]
    gyro_x = dataset['x-ang-accel'].values[i:i+window_size]
    gyro_y = dataset['y-ang-accel'].values[i:i+window_size]
    gyro_z = dataset['z-ang-accel'].values[i:i+window_size]
    total_label = dataset['action'].values[i:i+window_size]
    label = Counter(total_label).most_common()[0][0]
    sigmentation_data_temp.append([acc_x,acc_y,acc_z,gyro_x,gyro_y,gyro_z])
    sigmentation_arr = np.asarray(sigmentation_data_temp)
    sig_size = sigmentation_arr.shape
    if sig_size[2] == window_size:
      sigmentation_arr.reshape(window_size,6)
      sigmentation_data.append(sigmentation_arr)
      sigmentation_label.append(label)
      sigmentation_data_temp = []
    else:
      sigmentation_data_temp = []

  sigmentation_data_arr = np.asarray(sigmentation_data)
  sigmentation_label_arr = np.asarray(sigmentation_label)

  return sigmentation_data_arr,sigmentation_label_arr

In [54]:
from sklearn.model_selection import train_test_split

## For 128 window_size, with over lapping(75%, 50%, 25%, 0%)

In [94]:
#128 window_size

# 75% overlapping
feature_128wz_75ol, label_128wz_75ol = time_windows(window_size=128, overlapping=75, dataset=utd_df)
train_feature_128wz_75ol, test_feature_128wz_75ol,train_label_128wz_75ol, test_label_128wz_75ol = train_test_split(feature_128wz_75ol, label_128wz_75ol, test_size = 0.3)


# 50% overlapping
feature_128wz_50ol, label_128wz_50ol = time_windows(window_size=128, overlapping=50, dataset=utd_df)
train_feature_128wz_50ol, test_feature_128wz_50ol, train_label_128wz_50ol, test_label_128wz_50ol = train_test_split(feature_128wz_50ol, label_128wz_50ol, test_size = 0.3)

# 25% overlapping
feature_128wz_25ol, label_128wz_25ol = time_windows(window_size=128, overlapping=25, dataset=utd_df)
train_feature_128wz_25ol, test_feature_128wz_25ol, train_label_128wz_25ol, test_label_128wz_25ol = train_test_split(feature_128wz_25ol, label_128wz_25ol, test_size = 0.3)

# 0% overlapping
feature_128wz_0ol, label_128wz_0ol = time_windows(window_size=128, overlapping=0, dataset=utd_df)
train_feature_128wz_0ol, test_feature_128wz_0ol, train_label_128wz_0ol, test_label_128wz_0ol = train_test_split(feature_128wz_0ol, label_128wz_0ol, test_size = 0.3)

In [98]:
len(test_label_128wz_50ol)

729

In [102]:
np.save(f'../Data/UTDMHAD_data/UTD_train_feature_128wz_75ol.npy', train_feature_128wz_75ol)
np.save(f'../Data/UTDMHAD_data/UTD_train_label_128wz_75ol.npy', train_label_128wz_75ol)
np.save(f'../Data/UTDMHAD_data/UTD_test_feature_128wz_75ol.npy', test_feature_128wz_75ol)
np.save(f'../Data/UTDMHAD_data/UTD_test_label_128wz_75ol.npy', test_label_128wz_75ol)

np.save(f'../Data/UTDMHAD_data/UTD_train_feature_128wz_50ol.npy', train_feature_128wz_50ol)
np.save(f'../Data/UTDMHAD_data/UTD_train_label_128wz_50ol.npy', train_label_128wz_50ol)
np.save(f'../Data/UTDMHAD_data/UTD_test_feature_128wz_50ol.npy', test_feature_128wz_50ol)
np.save(f'../Data/UTDMHAD_data/UTD_test_label_128wz_50ol.npy', test_label_128wz_50ol)

np.save(f'../Data/UTDMHAD_data/UTD_train_feature_128wz_25ol.npy', train_feature_128wz_25ol)
np.save(f'../Data/UTDMHAD_data/UTD_train_label_128wz_25ol.npy', train_label_128wz_25ol)
np.save(f'../Data/UTDMHAD_data/UTD_test_feature_128wz_25ol.npy', test_feature_128wz_25ol)
np.save(f'../Data/UTDMHAD_data/UTD_test_label_128wz_25ol.npy', test_label_128wz_25ol)

np.save(f'../Data/UTDMHAD_data/UTD_train_feature_128wz_0ol.npy', train_feature_128wz_0ol)
np.save(f'../Data/UTDMHAD_data/UTD_train_label_128wz_0ol.npy', train_label_128wz_0ol)
np.save(f'../Data/UTDMHAD_data/UTD_test_feature_128wz_0ol.npy', test_feature_128wz_0ol)
np.save(f'../Data/UTDMHAD_data/UTD_test_label_128wz_0ol.npy', test_label_128wz_0ol)

In [89]:
print(len(train_feature_128wz_75ol))
print(len(train_feature_64wz_75ol))
print(len(train_feature_128wz_25ol))
print(len(train_feature_128wz_0ol))

3402
7778
1134
850


## For 64 window_size, with over lapping(75%, 50%, 25%, 0%)

In [90]:
#64 window_size

# 75% overlapping
feature_64wz_75ol, label_64wz_75ol = time_windows(window_size=64, overlapping=75, dataset=utd_df)
train_feature_64wz_75ol, test_feature_64wz_75ol, train_label_64wz_75ol, test_label_64wz_75ol = train_test_split(feature_64wz_75ol, label_64wz_75ol, test_size = 0.3)


# 50% overlapping
feature_64wz_50ol, label_64wz_50ol = time_windows(window_size=64, overlapping=50, dataset=utd_df)
train_feature_64wz_50ol, test_feature_64wz_50ol,train_label_64wz_50ol, test_label_64wz_50ol = train_test_split(feature_64wz_50ol, label_64wz_50ol, test_size = 0.3)

# 25% overlapping
feature_64wz_25ol, label_64wz_25ol = time_windows(window_size=64, overlapping=25, dataset=utd_df)
train_feature_64wz_25ol, test_feature_64wz_25ol,train_label_64wz_25ol, test_label_64wz_25ol = train_test_split(feature_64wz_25ol, label_64wz_25ol, test_size = 0.3)

# 0% overlapping
feature_64wz_0ol, label_64wz_0ol = time_windows(window_size=64, overlapping=0, dataset=utd_df)
train_feature_64wz_0ol, test_feature_64wz_0ol,train_label_64wz_0ol, test_label_64wz_0ol = train_test_split(feature_64wz_0ol, label_64wz_0ol, test_size = 0.3)

In [103]:

np.save(f'../Data/UTDMHAD_data/UTD_train_feature_64wz_75ol.npy', train_feature_64wz_75ol)
np.save(f'../Data/UTDMHAD_data/UTD_train_label_64wz_75ol.npy', train_label_64wz_75ol)
np.save(f'../Data/UTDMHAD_data/UTD_test_feature_64wz_75ol.npy', test_feature_64wz_75ol)
np.save(f'../Data/UTDMHAD_data/UTD_test_label_64wz_75ol.npy', test_label_64wz_75ol)

np.save(f'../Data/UTDMHAD_data/UTD_train_feature_64wz_50ol.npy', train_feature_64wz_50ol)
np.save(f'../Data/UTDMHAD_data/UTD_train_label_64wz_50ol.npy', train_label_64wz_50ol)
np.save(f'../Data/UTDMHAD_data/UTD_test_feature_64wz_50ol.npy', test_feature_64wz_50ol)
np.save(f'../Data/UTDMHAD_data/UTD_test_label_64wz_50ol.npy', test_label_64wz_50ol)

np.save(f'../Data/UTDMHAD_data/UTD_train_feature_64wz_25ol.npy', train_feature_64wz_25ol)
np.save(f'../Data/UTDMHAD_data/UTD_train_label_64wz_25ol.npy', train_label_64wz_25ol)
np.save(f'../Data/UTDMHAD_data/UTD_test_feature_64wz_25ol.npy', test_feature_64wz_25ol)
np.save(f'../Data/UTDMHAD_data/UTD_test_label_64wz_25ol.npy', test_label_64wz_25ol)

np.save(f'../Data/UTDMHAD_data/UTD_train_feature_64wz_0ol.npy', train_feature_64wz_0ol)
np.save(f'../Data/UTDMHAD_data/UTD_train_label_64wz_0ol.npy', train_label_64wz_0ol)
np.save(f'../Data/UTDMHAD_data/UTD_test_feature_64wz_0ol.npy', test_feature_64wz_0ol)
np.save(f'../Data/UTDMHAD_data/UTD_test_label_64wz_0ol.npy', test_label_64wz_0ol)

## For 32 window_size, with over lapping(75%, 50%, 25%, 0%)

In [92]:
#32 window_size

# 75% overlapping
feature_32wz_75ol, label_32wz_75ol = time_windows(window_size=32, overlapping=75, dataset=utd_df)
train_feature_32wz_75ol, test_feature_32wz_75ol,train_label_32wz_75ol, test_label_32wz_75ol = train_test_split(feature_32wz_75ol, label_32wz_75ol, test_size = 0.3)


# 50% overlapping
feature_32wz_50ol, label_32wz_50ol = time_windows(window_size=32, overlapping=50, dataset=utd_df)
train_feature_32wz_50ol, test_feature_32wz_50ol,train_label_32wz_50ol, test_label_32wz_50ol = train_test_split(feature_32wz_50ol, label_32wz_50ol, test_size = 0.3)

# 25% overlapping
feature_32wz_25ol, label_32wz_25ol = time_windows(window_size=32, overlapping=25, dataset=utd_df)
train_feature_32wz_25ol, test_feature_32wz_25ol,train_label_32wz_25ol, test_label_32wz_25ol = train_test_split(feature_32wz_25ol, label_32wz_25ol, test_size = 0.3)

# 0% overlapping
feature_32wz_0ol, label_32wz_0ol = time_windows(window_size=32, overlapping=0, dataset=utd_df)
train_feature_32wz_0ol, test_feature_32wz_0ol,train_label_32wz_0ol, test_label_32wz_0ol = train_test_split(feature_32wz_0ol, label_32wz_0ol, test_size = 0.3)

In [104]:
# save the dataset

np.save(f'../Data/UTDMHAD_data/UTD_train_feature_32wz_75ol.npy', train_feature_32wz_75ol)
np.save(f'../Data/UTDMHAD_data/UTD_train_label_32wz_75ol.npy', train_label_32wz_75ol)
np.save(f'../Data/UTDMHAD_data/UTD_test_feature_32wz_75ol.npy', test_feature_32wz_75ol)
np.save(f'../Data/UTDMHAD_data/UTD_test_label_32wz_75ol.npy', test_label_32wz_75ol)

np.save(f'../Data/UTDMHAD_data/UTD_train_feature_32wz_50ol.npy', train_feature_32wz_50ol)
np.save(f'../Data/UTDMHAD_data/UTD_train_label_32wz_50ol.npy', train_label_32wz_50ol)
np.save(f'../Data/UTDMHAD_data/UTD_test_feature_32wz_50ol.npy', test_feature_32wz_50ol)
np.save(f'../Data/UTDMHAD_data/UTD_test_label_32wz_50ol.npy', test_label_32wz_50ol)

np.save(f'../Data/UTDMHAD_data/UTD_train_feature_32wz_25ol.npy', train_feature_32wz_25ol)
np.save(f'../Data/UTDMHAD_data/UTD_train_label_32wz_25ol.npy', train_label_32wz_25ol)
np.save(f'../Data/UTDMHAD_data/UTD_test_feature_32wz_25ol.npy', test_feature_32wz_25ol)
np.save(f'../Data/UTDMHAD_data/UTD_test_label_32wz_25ol.npy', test_label_32wz_25ol)

np.save(f'../Data/UTDMHAD_data/UTD_train_feature_32wz_0ol.npy', train_feature_32wz_0ol)
np.save(f'../Data/UTDMHAD_data/UTD_train_label_32wz_0ol.npy', train_label_32wz_0ol)
np.save(f'../Data/UTDMHAD_data/UTD_test_feature_32wz_0ol.npy', test_feature_32wz_0ol)
np.save(f'../Data/UTDMHAD_data/UTD_test_label_32wz_0ol.npy', test_label_32wz_0ol)